<a href="https://colab.research.google.com/github/submouse9903/uos-deepLearning/blob/main/U47768_CH06_Transformer(Attention).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## torch version 2.0
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

### - Seq2Seq 에서 Q, K, V 의 역할
- query: output seq 
- key: input seq
- value: input seq

Attention은 embedded input sequence ($X_V$)를 output sequence ($X_Q$)와 input sequence ($X_K$), input seq의 다른 표현형)의 유사도를 이용하여 새로운 embedded input sequence를 출력한다.

- $X_Q \in \mathbb{R}^{S \times d_e} $: 길이가 $L$, 각 원소가 $d_e$차원에 embedding (내재화)됨.
- $X_K \in \mathbb{R}^{S \times d_k} $: 길이가 $S$, 각 원소가 $d_k$차원에 embedding (내재화)됨.
- $X_V \in \mathbb{R}^{S \times d_v} $: 길이가 $S$, 각 원소가 $d_v$차원에 embedding (내재화)됨.


다음을 만족한다. 
- $Q = X_Q W_Q^\top \in \mathbb{R}^{S \times d_e}$, 여기서 $W_Q \in \mathbb{R}^{d_e \times d_e}$
- $K = X_K W_K^\top \in \mathbb{R}^{S \times d_e}$, 여기서  $W_K \in \mathbb{R}^{d_e \times d_k}$
- $V = X_V W_V^\top \in \mathbb{R}^{S \times d_e}$, 여기서  $W_K \in \mathbb{R}^{d_e \times d_k}$

Attention weight 는 다음 값의 softmax  값으로 계산된다. 
$Q K^\top  =  X_Q W_Q^\top W_K X_K \in \mathbb{R}^{L \times S} $

Attention의 출력은 
$$ \mbox{softmax}(Q K^\top/\sqrt{d})V \in \mathbb{R}^{L \times d_e} $$
가 된다. 


**Attention 결론**

*Attention은 ''$d_v$ 차원에 embedding 된 길이 $S$의 input sequence'' 를 ''$d_e$ 차원에 embedding 된 길이 $L$의 output sequence'' 로 변환하는 seq2seq-layer다*


In [ ]:
# 변환후 feature dim
embed_dim = 4
# attention weight 를 계산할 input이 표현된 feature 차원
kdim = 7
# 현재 input이 표현되어 있는 feature의 차원  
vdim = 5

num_heads = 1
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True,
                                       kdim=kdim, vdim=vdim)

MultiheadAttention 의 경우 Attention 연산 후에 out proj 연산이 있다. 이 연산은 $d_e \times d_e$ 행렬에 의해 이루어진다.

In [ ]:
for i in multihead_attn.named_parameters():
  print(i)

('q_proj_weight', Parameter containing:
tensor([[ 0.8081,  0.8300,  0.3251,  0.6588],
        [ 0.7569, -0.8277,  0.2550,  0.0611],
        [-0.4685,  0.6590,  0.2442,  0.3335],
        [ 0.1542, -0.0311,  0.4335,  0.0395]], requires_grad=True))
('k_proj_weight', Parameter containing:
tensor([[ 0.0460, -0.3858,  0.3785, -0.3876, -0.3238,  0.3952, -0.2606],
        [-0.0296,  0.2311, -0.5357, -0.5317,  0.4605, -0.1561, -0.5099],
        [-0.3090,  0.3169,  0.5062,  0.0731, -0.7035,  0.6866, -0.6652],
        [ 0.3952,  0.5441, -0.7128,  0.0107, -0.0311, -0.0951,  0.0376]],
       requires_grad=True))
('v_proj_weight', Parameter containing:
tensor([[ 0.6255, -0.0111, -0.0438,  0.5052, -0.3811],
        [-0.6044,  0.1516, -0.2924, -0.0771,  0.3047],
        [ 0.6424, -0.8096,  0.0712,  0.6121,  0.5133],
        [ 0.1129,  0.2214, -0.1532,  0.3119,  0.6485]], requires_grad=True))
('in_proj_bias', Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=T

In [ ]:
# number of samples
n = 2
# output seq의 길이
L = 3
# input seq의 길이
S = 2
# data
X_q = torch.rand(n,L,embed_dim)
X_k = torch.rand(n,S,kdim)
X_v = torch.rand(n,S,vdim)

In [ ]:
attn_output, attn_output_weights = multihead_attn(X_q, X_k, X_v)

In [ ]:
attn_output_weights

tensor([[[0.3921, 0.6079],
         [0.4287, 0.5713],
         [0.4340, 0.5660]],

        [[0.4989, 0.5011],
         [0.4722, 0.5278],
         [0.4834, 0.5166]]], grad_fn=<MeanBackward1>)

- attn_output_weights 의 설명
  - input seq를  output seq 로 변환할 때, 각 output 변환값이 input 값의 어떤 구성요소로 이루어져있는지 비율로 표현
  - 현재는 input seq 의 길이가 2고 output seq의 길이가 3
  - 각 행은 output seq를 나타내며 행벡터의 원소값은 input seq 원소들의 가중치를 나타낸다. 



In [ ]:
attn_output

tensor([[[-0.1778,  0.0810,  0.2770, -0.3631],
         [-0.1738,  0.0801,  0.2782, -0.3670],
         [-0.1732,  0.0800,  0.2784, -0.3676]],

        [[ 0.0844, -0.2254,  0.2697, -0.2622],
         [ 0.0825, -0.2279,  0.2751, -0.2666],
         [ 0.0833, -0.2268,  0.2728, -0.2648]]], grad_fn=<TransposeBackward0>)

3행 4열 output을 얻게 되었는데, 이는 output length가 3이며 각 원소가 4차원에 embedding 된 것이다. 

### Masking 의 활용

- (L,S) 의 행렬이 필요함


In [ ]:
mask_mat = np.array([[1,-100], [1,-100],[1,1]])
mask_mat = torch.tensor(mask_mat, dtype = torch.float)

In [ ]:
attn_output, attn_output_weights = multihead_attn(X_q, X_k, X_v,
                                                  attn_mask = mask_mat)

In [ ]:
attn_output_weights

tensor([[[1.0000e+00, 2.1019e-44],
         [1.0000e+00, 1.8217e-44],
         [4.3401e-01, 5.6599e-01]],

        [[1.0000e+00, 1.4013e-44],
         [1.0000e+00, 1.5414e-44],
         [4.8345e-01, 5.1655e-01]]], grad_fn=<MeanBackward1>)

### Multihead Attention


In [ ]:
num_heads = 4
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True,
                                       kdim=kdim, vdim=vdim)

In [ ]:
for i,j in multihead_attn.named_parameters():
  print(i)
  print(j)

q_proj_weight
Parameter containing:
tensor([[ 0.6570,  0.5942, -0.4123,  0.6315],
        [-0.1148,  0.4508, -0.2153, -0.6365],
        [ 0.3044, -0.7152, -0.2825,  0.0821],
        [-0.6130,  0.8284,  0.1979, -0.2382]], requires_grad=True)
k_proj_weight
Parameter containing:
tensor([[ 0.1517,  0.1926, -0.5911,  0.4140, -0.3239, -0.2350, -0.2235],
        [ 0.0214, -0.4754, -0.4389, -0.1492,  0.4507, -0.2647, -0.1339],
        [ 0.2932, -0.1135, -0.1349, -0.0822, -0.6823, -0.3712,  0.3190],
        [ 0.0010,  0.1696,  0.2470, -0.6032,  0.3760, -0.6372, -0.6300]],
       requires_grad=True)
v_proj_weight
Parameter containing:
tensor([[-0.6550, -0.4997,  0.4274,  0.6898, -0.0595],
        [-0.3655,  0.6069, -0.3514, -0.7861, -0.2132],
        [-0.2290,  0.6656,  0.4749, -0.3340, -0.2303],
        [ 0.4406, -0.1715,  0.4626,  0.5878,  0.3684]], requires_grad=True)
in_proj_bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)
out_proj.weigh

### - Self Attention

In [ ]:
# 변환후 feature dim
embed_dim = 4
num_heads = 1
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)

In [ ]:
# number of samples
n = 2
# input seq의 길이
S = 5
# data
X_v = torch.rand(n,S,embed_dim)

In [ ]:
output, weight = multihead_attn(X_v, X_v, X_v)

In [ ]:
print("#### output of self attention")
output.shape

#### output of self attention


torch.Size([2, 5, 4])